In [3]:
from brian2 import *
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal
from scipy.fft import fft, fftfreq

def run_oscillation_regime(regime_params, duration=2000*ms, title=""):
    """
    Simulación optimizada para generar oscilaciones theta y gamma
    """
    start_scope()
    defaultclock.dt = 0.1*ms  # dt más grande para mejor resolución temporal
    
    N_exc = 800
    N_inh = 200
    N_total = N_exc + N_inh
    
    equations = '''
    dv/dt = (0.04*v**2 + 5*v + 140 - u + I_syn + I_thalamic + I_noise)/ms : 1
    du/dt = a*(b*v - u)/ms : 1 
    I_syn : 1
    I_thalamic: 1
    I_noise : 1
    a : 1
    b : 1  
    c : 1
    d : 1
    '''
    
    exc_neurons = NeuronGroup(N_exc, equations, 
                              threshold='v >= 30',
                              reset='v = c; u += d',
                              method='euler')
    
    inh_neurons = NeuronGroup(N_inh, equations,
                              threshold='v >= 30', 
                              reset='v = c; u += d',
                              method='euler')
    
    # Input talámico con modulación para theta
    thalamic_base_exc = regime_params.get('thalamic_base_exc', 2.0)
    thalamic_base_inh = regime_params.get('thalamic_base_inh', 1.0)
    theta_mod_strength = regime_params.get('theta_modulation', 0.5)
    theta_freq = regime_params.get('theta_frequency', 6.0)  # Hz
    
    noise_strength = regime_params.get('noise_strength', 0.1)
    
    @network_operation(dt=0.5*ms)
    def update_input():
        # Modulación theta del input talámico
        t_current = defaultclock.t
        theta_phase = 2 * np.pi * theta_freq * t_current/ms
        theta_modulation = 1 + theta_mod_strength * np.sin(theta_phase)
        
        # Input talámico modulado + ruido
        exc_neurons.I_thalamic = thalamic_base_exc * theta_modulation * np.random.poisson(1.5, N_exc)
        inh_neurons.I_thalamic = thalamic_base_inh * theta_modulation * np.random.poisson(1.0, N_inh)
        
        # Ruido independiente
        exc_neurons.I_noise = noise_strength * np.random.randn(N_exc)
        inh_neurons.I_noise = noise_strength * np.random.randn(N_inh)
    
    # Parámetros neuronales ajustados para favorecer oscilaciones
    r_exc = np.random.rand(N_exc)
    r_inh = np.random.rand(N_inh)
    
    # Excitatorias: más heterogeneidad para gamma
    exc_neurons.a = regime_params.get('exc_a', 0.02)
    exc_neurons.b = regime_params.get('exc_b', 0.2) + regime_params.get('exc_b_var', 0.05) * r_exc
    exc_neurons.c = regime_params.get('exc_c_base', -65) + regime_params.get('exc_c_var', 15) * r_exc**2
    exc_neurons.d = regime_params.get('exc_d_base', 8) + regime_params.get('exc_d_var', -6) * r_exc**2
    
    # Inhibitorias: parámetros para gamma rápido
    inh_neurons.a = regime_params.get('inh_a_base', 0.02) + regime_params.get('inh_a_var', 0.08) * r_inh
    inh_neurons.b = regime_params.get('inh_b_base', 0.25) + regime_params.get('inh_b_var', -0.05) * r_inh
    inh_neurons.c = regime_params.get('inh_c', -65)
    inh_neurons.d = regime_params.get('inh_d', 2)
    
    # Condiciones iniciales
    exc_neurons.v = -65 + 5*np.random.randn(N_exc) 
    exc_neurons.u = exc_neurons.b * exc_neurons.v
    inh_neurons.v = -65 + 5*np.random.randn(N_inh) 
    inh_neurons.u = inh_neurons.b * inh_neurons.v
    
    # Reset sináptico
    exc_neurons.run_regularly('I_syn = 0', when='before_synapses', dt=0.5*ms)
    inh_neurons.run_regularly('I_syn = 0', when='before_synapses', dt=0.5*ms)
    
    # Conectividad ajustada para oscilaciones
    k_exc = regime_params.get('k_exc', 3.0)  # Excitación más débil
    k_inh = regime_params.get('k_inh', 5.0)  # Inhibición más fuerte para gamma
    connectivity = regime_params.get('connectivity', 0.1)
    
    # Delays diferentes para cada tipo (importante para oscilaciones)
    delay_exc = regime_params.get('delay_exc', 1.0) * ms
    delay_inh = regime_params.get('delay_inh', 0.5) * ms  # Inhibición más rápida
    
    # Sinapsis excitatorias
    syn_ee = Synapses(exc_neurons, exc_neurons, 'w : 1', on_pre='I_syn += w')
    syn_ee.connect(p=connectivity)
    syn_ee.w = f'{k_exc} * rand()'
    syn_ee.delay = delay_exc
    
    syn_ei = Synapses(exc_neurons, inh_neurons, 'w : 1', on_pre='I_syn += w')
    syn_ei.connect(p=connectivity * 1.5)  # Más conexiones E->I
    syn_ei.w = f'{k_exc * 1.2} * rand()'
    syn_ei.delay = delay_exc
    
    # Sinapsis inhibitorias (clave para gamma)
    syn_ie = Synapses(inh_neurons, exc_neurons, 'w : 1', on_pre='I_syn += w')
    syn_ie.connect(p=connectivity * 2.0)  # Más inhibición
    syn_ie.w = f'-{k_inh} * rand()'
    syn_ie.delay = delay_inh
    
    syn_ii = Synapses(inh_neurons, inh_neurons, 'w : 1', on_pre='I_syn += w')
    syn_ii.connect(p=connectivity * 0.8)
    syn_ii.w = f'-{k_inh * 0.8} * rand()'
    syn_ii.delay = delay_inh
    
    # Monitores
    spike_mon_exc = SpikeMonitor(exc_neurons)
    spike_mon_inh = SpikeMonitor(inh_neurons)
    
    # Monitor de población para LFP simulado
    pop_mon_exc = PopulationRateMonitor(exc_neurons)
    pop_mon_inh = PopulationRateMonitor(inh_neurons)
    
    print(f"Simulando régimen: {title}")
    run(duration)
    
    return {
        'spike_mon_exc': spike_mon_exc,
        'spike_mon_inh': spike_mon_inh,
        'pop_mon_exc': pop_mon_exc,
        'pop_mon_inh': pop_mon_inh,
        'N_exc': N_exc,
        'N_total': N_total,
        'title': title,
        'duration': duration
    }

def analyze_spectrum(results, plot=True):
    """
    Análisis espectral detallado de la actividad poblacional
    """
    # Usar tasa poblacional como proxy del LFP
    pop_rate = results['pop_mon_exc'].rate/Hz
    dt = results['pop_mon_exc'].t[1] - results['pop_mon_exc'].t[0]
    fs = 1.0 / (dt/second)
    
    # Calcular espectro de potencias usando Welch
    freqs, psd = signal.welch(pop_rate, fs, nperseg=min(1024, len(pop_rate)//4))
    
    # Encontrar picos en bandas de interés
    theta_band = (freqs >= 4) & (freqs <= 8)
    alpha_band = (freqs >= 8) & (freqs <= 12)
    gamma_band = (freqs >= 30) & (freqs <= 100)
    
    theta_power = np.max(psd[theta_band]) if np.any(theta_band) else 0
    alpha_power = np.max(psd[alpha_band]) if np.any(alpha_band) else 0
    gamma_power = np.max(psd[gamma_band]) if np.any(gamma_band) else 0
    
    theta_freq_peak = freqs[theta_band][np.argmax(psd[theta_band])] if np.any(theta_band) else 0
    gamma_freq_peak = freqs[gamma_band][np.argmax(psd[gamma_band])] if np.any(gamma_band) else 0
    
    # Calcular ratios
    theta_gamma_ratio = theta_power / gamma_power if gamma_power > 0 else 0
    gamma_alpha_ratio = gamma_power / alpha_power if alpha_power > 0 else 0
    
    if plot:
        plt.figure(figsize=(12, 8))
        
        # Subplot 1: Espectro completo
        plt.subplot(2, 2, 1)
        plt.semilogy(freqs, psd, 'k-', linewidth=1)
        plt.axvspan(4, 8, alpha=0.3, color='green', label='Theta (4-8 Hz)')
        plt.axvspan(8, 12, alpha=0.3, color='blue', label='Alpha (8-12 Hz)')
        plt.axvspan(30, 100, alpha=0.3, color='red', label='Gamma (30-100 Hz)')
        plt.xlabel('Frecuencia (Hz)')
        plt.ylabel('PSD')
        plt.title(f'Espectro de Potencias - {results["title"]}')
        plt.legend()
        plt.xlim(0, 100)
        
        # Subplot 2: Zoom en theta
        plt.subplot(2, 2, 2)
        plt.plot(freqs[theta_band], psd[theta_band], 'g-', linewidth=2)
        plt.axvline(theta_freq_peak, color='green', linestyle='--', alpha=0.7)
        plt.xlabel('Frecuencia (Hz)')
        plt.ylabel('PSD')
        plt.title(f'Banda Theta (Pico: {theta_freq_peak:.1f} Hz)')
        
        # Subplot 3: Zoom en gamma
        plt.subplot(2, 2, 3)
        plt.plot(freqs[gamma_band], psd[gamma_band], 'r-', linewidth=2)
        plt.axvline(gamma_freq_peak, color='red', linestyle='--', alpha=0.7)
        plt.xlabel('Frecuencia (Hz)')
        plt.ylabel('PSD')
        plt.title(f'Banda Gamma (Pico: {gamma_freq_peak:.1f} Hz)')
        
        # Subplot 4: Actividad temporal
        plt.subplot(2, 2, 4)
        t_plot = results['pop_mon_exc'].t/ms
        plt.plot(t_plot, pop_rate, 'k-', alpha=0.7)
        plt.xlabel('Tiempo (ms)')
        plt.ylabel('Tasa Poblacional (Hz)')
        plt.title('Actividad Poblacional')
        
        plt.tight_layout()
        plt.show()
    
    return {
        'freqs': freqs,
        'psd': psd,
        'theta_power': theta_power,
        'gamma_power': gamma_power,
        'alpha_power': alpha_power,
        'theta_freq_peak': theta_freq_peak,
        'gamma_freq_peak': gamma_freq_peak,
        'theta_gamma_ratio': theta_gamma_ratio,
        'gamma_alpha_ratio': gamma_alpha_ratio
    }

In [4]:
# REGÍMENES OPTIMIZADOS PARA THETA Y GAMMA

# 1. RÉGIMEN THETA FUERTE
regime_theta_strong = {
    'thalamic_base_exc': 1.5,
    'thalamic_base_inh': 0.8,
    'theta_modulation': 0.8,  # Modulación fuerte
    'theta_frequency': 6.0,
    'noise_strength': 0.05,
    'k_exc': 2.5,
    'k_inh': 4.0,
    'connectivity': 0.08,
    'delay_exc': 2.0,
    'delay_inh': 1.0,
    'exc_b_var': 0.02,  # Menos heterogeneidad para theta
}

# 2. RÉGIMEN GAMMA FUERTE  
regime_gamma_strong = {
    'thalamic_base_exc': 3.0,
    'thalamic_base_inh': 2.0,
    'theta_modulation': 0.2,  # Modulación débil
    'theta_frequency': 6.0,
    'noise_strength': 0.15,
    'k_exc': 2.0,
    'k_inh': 6.0,  # Inhibición fuerte para gamma
    'connectivity': 0.15,
    'delay_exc': 1.0,
    'delay_inh': 0.3,  # Inhibición muy rápida
    'exc_b_var': 0.08,  # Más heterogeneidad
    'inh_a_var': 0.1,   # Inhibitorias más rápidas
}

# 3. RÉGIMEN THETA-GAMMA ACOPLADO
regime_theta_gamma_coupled = {
    'thalamic_base_exc': 2.2,
    'thalamic_base_inh': 1.5,
    'theta_modulation': 0.6,
    'theta_frequency': 7.0,
    'noise_strength': 0.1,
    'k_exc': 2.8,
    'k_inh': 5.2,
    'connectivity': 0.12,
    'delay_exc': 1.5,
    'delay_inh': 0.6,
    'exc_b_var': 0.05,
    'inh_a_var': 0.06,
}

# 4. RÉGIMEN OPTIMIZADO (basado en literatura)
regime_optimized = {
    'thalamic_base_exc': 2.0,
    'thalamic_base_inh': 1.2,
    'theta_modulation': 0.5,
    'theta_frequency': 6.5,
    'noise_strength': 0.08,
    'k_exc': 2.2,
    'k_inh': 5.5,
    'connectivity': 0.11,
    'delay_exc': 1.2,
    'delay_inh': 0.4,
    'exc_a': 0.02,
    'exc_b': 0.22,
    'exc_b_var': 0.03,
    'inh_a_base': 0.1,  # Inhibitorias más rápidas
    'inh_a_var': 0.05,
    'inh_b_base': 0.3,
}

print("=== OPTIMIZACIÓN PARA OSCILACIONES THETA Y GAMMA ===\n")

# Ejecutar simulaciones
regimes = [
    (regime_theta_strong, "Theta Fuerte"),
    (regime_gamma_strong, "Gamma Fuerte"), 
    (regime_theta_gamma_coupled, "Theta-Gamma Acoplado"),
    (regime_optimized, "Optimizado")
]

results_list = []
spectrum_results = []

for regime_params, title in regimes[:1]:
    print(f"\n--- Simulando: {title} ---")
    results = run_oscillation_regime(regime_params, duration=3000*ms, title=title)
    spectrum = analyze_spectrum(results, plot=True)
    
    results_list.append(results)
    spectrum_results.append(spectrum)
    
    print(f"Theta: {spectrum['theta_freq_peak']:.1f} Hz (Potencia: {spectrum['theta_power']:.1f})")
    print(f"Gamma: {spectrum['gamma_freq_peak']:.1f} Hz (Potencia: {spectrum['gamma_power']:.1f})")
    print(f"Ratio Theta/Gamma: {spectrum['theta_gamma_ratio']:.2f}")

# Comparación final
print("\n=== RESUMEN COMPARATIVO ===")
print("Régimen\t\t\tTheta (Hz)\tGamma (Hz)\tRatio T/G")
print("-" * 60)
for i, (_, title) in enumerate(regimes):
    spec = spectrum_results[i]
    print(f"{title:20s}\t{spec['theta_freq_peak']:6.1f}\t\t{spec['gamma_freq_peak']:6.1f}\t\t{spec['theta_gamma_ratio']:6.2f}")

=== OPTIMIZACIÓN PARA OSCILACIONES THETA Y GAMMA ===


--- Simulando: Theta Fuerte ---


KeyboardInterrupt: 